In [1]:
import os
import zipfile
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import (
    ResNet152, MobileNetV3Large, VGG19, Xception
)


C:\Users\atish\AppData\Roaming\Python\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:
ZIP_PATH = "alzehmier.zip"   # change if needed

WORKING_DIR = "ALZ_working"
DATA_DIR = os.path.join(WORKING_DIR, "dataset")
MIXED_DIR = os.path.join(WORKING_DIR, "mixed_images")

os.makedirs(WORKING_DIR, exist_ok=True)
os.makedirs(MIXED_DIR, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(WORKING_DIR)

print("✅ Dataset unzipped completely")


✅ Dataset unzipped completely


In [5]:
image_records = []
image_counter = 0

dataset_root = DATA_DIR
print("📂 Dataset root detected:", dataset_root)

for class_name in sorted(os.listdir(dataset_root)):
    class_path = os.path.join(dataset_root, class_name)

    if not os.path.isdir(class_path):
        continue

    print(f"➡️ Processing class: {class_name}")

    for image_name in sorted(os.listdir(class_path)):
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):

            image_counter += 1
            new_image_name = f"img_{image_counter:06d}.png"

            src_path = os.path.join(class_path, image_name)
            dst_path = os.path.join(MIXED_DIR, new_image_name)

            shutil.copy(src_path, dst_path)

            image_records.append({
                "image_name": new_image_name,
                "label": class_name
            })

print(f"✅ Mixed images created: {image_counter}")

labels_df = pd.DataFrame(image_records)
LABELS_CSV = os.path.join(WORKING_DIR, "labels.csv")
labels_df.to_csv(LABELS_CSV, index=False)

print("✅ labels.csv saved successfully")


📂 Dataset root detected: ALZ_working\dataset
➡️ Processing class: Mild_Demented
➡️ Processing class: Moderate_Demented
➡️ Processing class: Non_Demented
➡️ Processing class: Very_Mild_Demented
✅ Mixed images created: 6400
✅ labels.csv saved successfully


In [6]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(rescale=1./255)

generator = datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=MIXED_DIR,
    x_col="image_name",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)


Found 6400 validated image filenames belonging to 4 classes.


In [7]:
class_counts = labels_df['label'].value_counts()
total = class_counts.sum()

class_weights = {
    i: total / (len(class_counts) * count)
    for i, count in enumerate(class_counts)
}

print("⚖️ Class Weights:", class_weights)


⚖️ Class Weights: {0: np.float64(0.5), 1: np.float64(0.7142857142857143), 2: np.float64(1.7857142857142858), 3: np.float64(25.0)}


In [8]:
def build_model(base_model, num_classes):
    base = base_model(
        include_top=False,
        weights=None,
        input_shape=(224, 224, 3)
    )

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(256, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=output)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model

FEATURE_DIR = "saved_features"
os.makedirs(FEATURE_DIR, exist_ok=True)

true_labels = labels_df['label'].astype('category').cat.codes.values
NUM_CLASSES = labels_df['label'].nunique()

print("✅ Common setup completed")


In [ ]:
feature_results = {}
generator.shuffle = False


In [ ]:
print("\n🚀 Training EfficientNetB0")

efficientnet = build_model(tf.keras.applications.EfficientNetB0, NUM_CLASSES)

efficientnet.fit(
    generator,
    epochs=10,
    class_weight=class_weights,
    verbose=1
)

efficientnet_fe = Model(
    inputs=efficientnet.input,
    outputs=efficientnet.layers[-3].output
)

efficientnet_features = efficientnet_fe.predict(generator)

np.save(
    os.path.join(FEATURE_DIR, "EfficientNetB0_features.npy"),
    efficientnet_features
)

feature_results["EfficientNetB0"] = efficientnet_features

print("💾 EfficientNetB0 features saved:", efficientnet_features.shape)



🚀 Training ResNet152
Epoch 1/5


In [ ]:
print("\n🚀 Training ResNet152")

resnet = build_model(tf.keras.applications.ResNet152, NUM_CLASSES)

resnet.fit(
    generator,
    epochs=10,
    class_weight=class_weights,
    verbose=1
)

resnet_fe = Model(
    inputs=resnet.input,
    outputs=resnet.layers[-3].output
)

resnet_features = resnet_fe.predict(generator)

np.save(
    os.path.join(FEATURE_DIR, "ResNet152_features.npy"),
    resnet_features
)

feature_results["ResNet152"] = resnet_features

print("💾 ResNet152 features saved:", resnet_features.shape)


In [ ]:
print("\n🚀 Training MobileNetV3")

mobilenet = build_model(
    tf.keras.applications.MobileNetV3Large,
    NUM_CLASSES
)

mobilenet.fit(
    generator,
    epochs=10,
    class_weight=class_weights,
    verbose=1
)

mobilenet_fe = Model(
    inputs=mobilenet.input,
    outputs=mobilenet.layers[-3].output
)

mobilenet_features = mobilenet_fe.predict(generator)

np.save(
    os.path.join(FEATURE_DIR, "MobileNetV3_features.npy"),
    mobilenet_features
)

print("💾 MobileNetV3 features saved:", mobilenet_features.shape)










In [ ]:
print("\n🚀 Training VGG19")

vgg = build_model(tf.keras.applications.VGG19, NUM_CLASSES)

vgg.fit(
    generator,
    epochs=10,
    class_weight=class_weights,
    verbose=1
)

vgg_fe = Model(
    inputs=vgg.input,
    outputs=vgg.layers[-3].output
)

vgg_features = vgg_fe.predict(generator)

np.save(
    os.path.join(FEATURE_DIR, "VGG19_features.npy"),
    vgg_features
)

print("💾 VGG19 features saved:", vgg_features.shape)

In [ ]:
print("\n🚀 Training Xception")

xception = build_model(tf.keras.applications.Xception, NUM_CLASSES)

xception.fit(
    generator,
    epochs=10,
    class_weight=class_weights,
    verbose=1
)

xception_fe = Model(
    inputs=xception.input,
    outputs=xception.layers[-3].output
)

xception_features = xception_fe.predict(generator)

np.save(
    os.path.join(FEATURE_DIR, "Xception_features.npy"),
    xception_features
)

print("💾 Xception features saved:", xception_features.shape)

In [ ]:
np.save(os.path.join(FEATURE_DIR, "true_labels.npy"), true_labels)


In [ ]:
results = []

for model_name, features in feature_results.items():
    print(f"\n🔍 Clustering for {model_name}")

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    pca = PCA(n_components=0.95)
    features_pca = pca.fit_transform(features_scaled)

    kmeans = KMeans(n_clusters=NUM_CLASSES, random_state=42)
    cluster_labels = kmeans.fit_predict(features_pca)

    changed = np.sum(cluster_labels != true_labels)
    change_percentage = (changed / len(true_labels)) * 100

    results.append({
        "Model": model_name,
        "Original Samples": len(true_labels),
        "Changed Labels": changed,
        "Change %": round(change_percentage, 2)
    })


In [ ]:
results_df = pd.DataFrame(results)
print("\n📊 FINAL CLUSTERING RESULTS")
display(results_df)
